### Options Pricer

Saurav Luthra, Dec 2021

My goal is to calculate the price of options along a certain stock's option chain using the Black-Scholes Model and Binomial Method, and eventually scrape real-time market data to compare real bid/ask prices to calculated prices. The stock whose options chain is modeled will be user selected, as well as the expiry date for the option chain.

I'm hoping this options pricer project will lead me to a more in-depth understanding of options pricing and volatility, as well as the technical tools needed to create an algorithmic trading strategy from scratch. This project will not account for transaction costs and will not be able to execute trades on its own (due to some external restrictions) however I hope to implement such features in the future.

In [31]:
from math import log, sqrt, pi, exp
import sys
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import os
import pandas_datareader as web
from math import *
import matplotlib

In [45]:
def norm_cdf(x):
    #'Cumulative distribution function for the standard normal distribution'
    #https://stackoverflow.com/questions/809362/how-to-calculate-cumulative-normal-distribution
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def norm_pdf(x, mean, sd):
     #'Probability Density function for the standard normal distribution'
    #https://stackoverflow.com/questions/12412895/how-to-calculate-probability-in-a-normal-distribution-given-mean-standard-devi
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

# S - spot price, K - 
def d1 (S,K,T,r,sigma):
    return(log(S/K) + (r + sigma**2/2.) * T) / (sigma * sqrt(T))

def d2 (S,K,T,r,sigma):
    return d1(S,K,T,r,sigma) - sigma * sqrt(T)

def bs_call (S,K,T,r,sigma):
    return S * norm_cdf(d1(S,K,T,r,sigma)) - K * exp(-r * T) * norm_cdf(d2(S,K,T,r,sigma))
  
def bs_put (S,K,T,r,sigma):
    return K * exp(-r * T) - S + bs_call(S,K,T,r,sigma)

def call_delta(S,K,T,r,sigma):
    return norm_cdf(d1(S,K,T,r,sigma))

def call_gamma(S,K,T,r,sigma):
    return norm_pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))

def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm_pdf(d1(S,K,T,r,sigma))*sqrt(T))

def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm_pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm_cdf(d2(S,K,T,r,sigma)))
    
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm_cdf(d2(S,K,T,r,sigma)))
    
def put_delta(S,K,T,r,sigma):
    return -norm_cdf(-d1(S,K,T,r,sigma))

def put_gamma(S,K,T,r,sigma):
    return norm_pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))

def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm_pdf(d1(S,K,T,r,sigma))*sqrt(T))

def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm_pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm_cdf(-d2(S,K,T,r,sigma)))

def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm_cdf(-d2(S,K,T,r,sigma)))

In [69]:
stock = input("STOCK:")
print("Ticker: ", stock)

Ticker:  AAPL


In [97]:
strike_price = input("STRIKE PRICE:")
print("Strike Price: $", strike_price)

Strike Price: $ 170


In [94]:
expiry = '01-14-2022' #MM-DD-YYYY


today = datetime.now()
yesterday = today.replace(day=today.day-1)
one_year_ago = today.replace(year=today.year-1)

print("Today, Yesterday, One Yr Ago:", today, yesterday, one_year_ago)

Today, Yesterday, One Yr Ago: 2022-01-07 20:56:17.467199 2022-01-06 20:56:17.467199 2021-01-07 20:56:17.467199


In [99]:
df = web.DataReader(stock, 'yahoo', one_year_ago, today)

df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

sigma = np.sqrt(252) * df['returns'].std()
uty = (web.DataReader("^TNX", 'yahoo', yesterday, today))#['Close'].iloc[-1])/100

tnx_yield = (uty.iloc[0]['Close'])/100
#lcp = df['Close'].iloc[-1]
#t = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days / 365

#print('The Option Price according to B-S model is: $', bs_call(lcp, strike_price, t, uty, sigma))

spot_price = df.iloc[-1]['Close']


In [100]:
spot_price

172.1699981689453

In [36]:
def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm_cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm_cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

print("Implied Volatility: " +
      str(100 * call_implied_volatility(bs_call(lcp, strike_price, t, uty, sigma,), lcp, strike_price, t, uty,)) + " %")

Implied Volatility: 0.1 %


In [91]:
#S-spot price, K-strike price, T-time until expiry, r-risk free rate
#sigma-past year returns volatility, n-num of time steps, cp-call or put, ae-american or european 
def CRR_binomial_tree(S,K,T,r,sigma,n,cp,ae, div):

    u = up_movement(sigma, T, n)
    d = down_movement(u)
    p_up = prob_u(r, T, n, d, u)
    p_down = 1-p_up
    
    stockTree = stock_price_tree(S, u, d, n)
    probTree = probability_tree (n, p_up, p_down)

    if ae == 'e':
        payoffTree = european_payoff_tree(stockTree, n, K, cp)
    elif ae == 'a':
        payoffTree = american_payoff_tree(stockTree, n, K, cp)

    optionPrice = backwardsDiscount(K, ae, cp, p_up, p_down, n, r, T, div, payoffTree, probTree, stockTree)

    print("\nSTOCKTREE  \n", stockTree)
    print("\nPROBTREE   \n", probTree)
    print("\nPAYOFFTREE \n", payoffTree)
    print("\nOPTPRICE   \n", optionPrice)
    return optionPrice


def up_movement(sigma, T, n):

    return exp(sigma * sqrt(T/n))

def down_movement(u):

    return 1/u

def prob_u(r, T, n, d, u):

    return exp(r * (T/n) - d) / (u - d)

def stock_price_tree(S, u ,d, n):
    stocktree = np.zeros((n+1, n+1))
    for j in range(n+1):
        for i in range(j+1):
            stocktree[i][j] = S * (u**i) * (d**(j - i))

    return stocktree

def probability_tree(n, p_up, p_down):
    probtree = np.zeros((n+1, n+1))
    for j in range(n+1):
        for i in range(j+1):
            probtree[i][j] = factorial(j) / (factorial(i) * factorial(j - i)) * (p_up**i) * (p_down**(j - i))
    
    return probtree

def european_payoff_tree(stocktree, n, K, cp):
    #Initialize all payoff nodes to zeros
    payoffTree = np.zeros((n+1, n+1))
    iopt = 0
    # Get payoffs at terminal nodes at the option maturity
    if cp == 'c':
        iopt = 1
    else:
        iopt = -1
    # Get payoffs at terminal nodes
    for i in range(n+1):
        payoffTree[i][n] = np.maximum(0, iopt * (stocktree[i][n]-K))

    return payoffTree

def american_payoff_tree(stocktree, n, K, cp):
    #Initialize all payoff nodes to zeros
    payoffTree = np.zeros((n+1, n+1))
    iopt = 0
    if cp == 'c':
        iopt = 1
    else:
        iopt = -1
    # Get payoffs at each node
    for j in range(n+1):
        for i in range(j+1):
            payoffTree[i][j] = np.maximum(0, iopt * (stocktree[i][j]-K))

    return payoffTree

def backwardsDiscount(K, ae, cp, p_up, p_down, n, r, T, div, payoffTree, probTree, stockTree):
    optionprice = 0.0
    if ae == 'e':
        TerminalPayoff = 0.0
        for i in range(n+1):
            TerminalPayoff = TerminalPayoff + payoffTree[i][n] * probTree[i][n]
     
        optionprice = TerminalPayoff * np.exp(-r * T)
    else:
    # Step backward through tree
        if cp == 'c':
            iopt = 1
        else:
            iopt = -1
        for j in range(n-1,-1,-1): 
            for i in range(j+1):
                payoffTree[i][j] = (p_up * payoffTree[i+1][j+1] + p_down * payoffTree[i][j+1]) * np.exp(-1 * (r - div) * (T/n))
                # Use Early exercise price
                payoffTree[i][j] = np.maximum(iopt * (stockTree[i][j] - K), payoffTree[i][j])  
   
        optionprice = payoffTree[0][0]

    return optionprice


In [103]:
#S-spot price, K-strike price, T-time until expiry, r-risk free rate
#sigma-past year returns volatility, n-num of time steps, cp-call or put, ae-american or european 
print("SPOT PRICE: $", spot_price)
CRR_binomial_tree(spot_price,  int(strike_price), 5 , tnx_yield, sigma, 5, 'c', 'a', 0)

SPOT PRICE: $ 172.1699981689453

STOCKTREE  
 [[172.16999817 134.29368407 104.74992027  81.70559823  63.73088176
   49.71048958]
 [  0.         220.72898272 172.16999817 134.29368407 104.74992027
   81.70559823]
 [  0.           0.         282.9835879  220.72898272 172.16999817
  134.29368407]
 [  0.           0.           0.         362.79653916 282.9835879
  220.72898272]
 [  0.           0.           0.           0.         465.12000856
  362.79653916]
 [  0.           0.           0.           0.           0.
  596.30288333]]

PROBTREE   
 [[1.00000000e+00 7.05944559e-02 4.98357721e-03 3.51812921e-04
  2.48360418e-05 1.75328685e-06]
 [0.00000000e+00 9.29405544e-01 1.31221757e-01 1.38952929e-02
  1.30790752e-03 1.15413775e-04]
 [0.00000000e+00 0.00000000e+00 8.63794665e-01 1.82937343e-01
  2.58287244e-02 3.03894125e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 8.02815551e-01
  2.26697308e-01 4.00089328e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  7.4614

318.9060563699812